In [1]:
import pandas as pd
import numpy as np
import gc
import zipfile
import pip
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from yellowbrick.classifier import ClassificationReport
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [4]:
# reading in the train and testing data
zf = zipfile.ZipFile('microsoft-malware-prediction.zip')

train = pd.read_csv(zf.open('train.csv'), dtype=dtypes)
test = pd.read_csv(zf.open('test.csv'), dtype=dtypes)
df_sample = pd.read_csv(zf.open('sample_submission.csv'), nrows=100)

In [7]:
print(train.shape)
print(test.shape)

(8921483, 83)
(7853253, 82)


## Combining Train and Test to explore which features to remove

In [5]:
frames = [train, test]
df = pd.concat(frames)
df.shape

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(16774736, 83)

## Cleaning the Data

In [5]:
# dictionary to keep track of features that I can drop in order to down-size
remove_features = []

In [6]:
features = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_df_data = pd.concat([features, percent], axis=1, keys=['Total', 'Percent'])
missing_df_data.head(5)

,Total,Percent
PuaMode,16770239,99.973192
Census_ProcessorClass,16719874,99.672949
DefaultBrowsersIdentifier,16034179,95.585284
Census_IsFlightingInternal,14082721,83.951968
Census_InternalBatteryType,12318273,73.433484


In [7]:
# dropping PuaMode and Census_ProcessorClass features because 99% of those values are mising
remove_features.append('PuaMode')
remove_features.append('Census_ProcessorClass')

## Skewed Columns

In [8]:
pd.options.display.float_format = '{:,.2f}'.format
sk_df = pd.DataFrame([{'column': c, 'unique': df[c].nunique(), 'skewness': df[c].value_counts(normalize=True).values[0] * 100} for c in df.columns])
sk_df = sk_df.sort_values('skewness', ascending=False)
sk_df

,column,skewness,unique
27,Census_IsWIMBootEnabled,100.00,2
61,IsBeta,100.00,2
21,Census_IsFlightsDisabled,100.00,2
20,Census_IsFlightingInternal,100.00,2
4,AutoSampleOptIn,100.00,2
50,Census_ThresholdOptIn,99.97,2
23,Census_IsPortableOperatingSystem,99.94,2
77,SMode,99.90,2
8,Census_DeviceFamily,99.85,3
75,PuaMode,99.73,2


In [9]:
remove_features.extend(sk_df[sk_df.skewness > 99].column.tolist())
remove_features

['PuaMode',
 'Census_ProcessorClass',
 'Census_IsWIMBootEnabled',
 'IsBeta',
 'Census_IsFlightsDisabled',
 'Census_IsFlightingInternal',
 'AutoSampleOptIn',
 'Census_ThresholdOptIn',
 'Census_IsPortableOperatingSystem',
 'SMode',
 'Census_DeviceFamily',
 'PuaMode',
 'UacLuaenable',
 'Census_IsVirtualDevice',
 'ProductName']

In [10]:
# PuaMode is duplicated in the two categories, so dropping it once in the dictionary of 
# features to be dropped in training and testing datasets
remove_features.remove('PuaMode')

# dropping these columns.
train.drop(remove_features, axis=1, inplace=True)
test.drop(remove_features, axis=1, inplace=True)

In [11]:
# dropping these values in the big dataframe to speed up this process 
df.drop(remove_features, axis=1, inplace=True)
df.drop('MachineIdentifier', axis=1, inplace=True)
df.drop('HasDetections', axis=1, inplace=True)

## Imputing/Filling in missing values more than 5%

In [12]:
null_values = df.isnull().sum()
null_values = null_values / df.shape[0]
null_values[null_values > 0.05]

Census_InternalBatteryType   0.73
DefaultBrowsersIdentifier    0.96
OrganizationIdentifier       0.31
SmartScreen                  0.40
dtype: float64

In [13]:
# Census_InternalBatteryType
pd.options.display.max_rows = 99
df.Census_InternalBatteryType.value_counts()

lion        3457138
li-i         420382
#            359305
lip          109179
liio          57259
li p          14683
li            11548
nimh           8042
real           4859
bq20           4220
pbac           3895
vbox           2630
unkn            928
lgi0            635
lipo            357
lhp0            273
4cel            262
ithi            138
lipp            127
batt            125
virt             69
ram              66
bad              58
pad0             39
ca48             29
lit              27
a132             17
ots0             15
ÿÿÿÿ             11
lio              9
lai0              9
4lio              9
li-p              8
icp3              8
lio               8
5nm1              6
0x0b              5
h00j              5
lgs0              4
0ts0              4
asmb              4
a138              4
a140              3
#TAB#             3
3ion              2
l
?               2
3500              2
li-l              2
lhpo              2
6ion              2


In [14]:
trans_dict = {
    '˙˙˙': 'unknown', 'unkn': 'unknown', np.nan: 'unknown'
}
train.replace({'Census_InternalBatteryType': trans_dict}, inplace=True)
test.replace({'Census_InternalBatteryType': trans_dict}, inplace=True)

In [15]:
# DefaultBrowsersIdentifier

# Replacing missing values with 0 in both training and testing 
df.DefaultBrowsersIdentifier.value_counts().head(5)
train.DefaultBrowsersIdentifier.fillna(0, inplace=True)
test.DefaultBrowsersIdentifier.fillna(0, inplace=True)

In [16]:
# OrganizationIdentifier

# Replacing missing values with 0 in both training and testing 
df.OrganizationIdentifier.value_counts()
train.replace({'OrganizationIdentifier': {np.nan: 0}}, inplace=True)
test.replace({'OrganizationIdentifier': {np.nan: 0}}, inplace=True)

In [17]:
# SmartScreen

df.SmartScreen.value_counts()

RequireAdmin     7729743
ExistsNotSet     1646629
Off               347924
Warn              261408
Prompt             63422
Block              43774
off                 3133
On                  1536
&#x02;               820
&#x01;               602
on                   280
requireadmin          20
OFF                   11
0                      6
warn                   2
Promt                  2
00000000               1
Deny                   1
&#x03;                 1
RequiredAdmin          1
requireAdmin           1
Promprt                1
Enabled                1
of                     1
BLOCK                  1
prompt                 1
ON                     1
Name: SmartScreen, dtype: int64

In [18]:
trans_dict = {
    'off': 'Off', '&#x02;': '2', '&#x01;': '1', 'on': 'On', 'requireadmin': 'RequireAdmin', 'OFF': 'Off', 
    'Promt': 'Prompt', 'requireAdmin': 'RequireAdmin', 'prompt': 'Prompt', 'warn': 'Warn', 
    '00000000': '0', '&#x03;': '3', np.nan: 'NaN'
}
train.replace({'SmartScreen': trans_dict}, inplace=True)
test.replace({'SmartScreen': trans_dict}, inplace=True)

In [19]:
train.drop('MachineIdentifier', axis=1, inplace=True)

In [20]:
# saving to appened back to training later 
test_MachineIdentifier = test['MachineIdentifier']
test.drop('MachineIdentifier', axis=1, inplace=True)

In [21]:
train.dropna(inplace=True)
train.shape

(7667789, 68)

In [22]:
test.dropna(inplace=True)
test.shape

(6800765, 67)

## Reducing Memory Usage

In [23]:
# code from: https://www.kaggle.com/timon88/load-whole-data-without-any-dtypes

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [25]:
# label encoding for categorical columns
train['SmartScreen'] = train.SmartScreen.astype('category')
train['Census_InternalBatteryType'] = train.Census_InternalBatteryType.astype('category')
cate_cols_train = train.select_dtypes(include='category').columns.tolist()

test['SmartScreen'] = test.SmartScreen.astype('category')
test['Census_InternalBatteryType'] = test.Census_InternalBatteryType.astype('category')
cate_cols_test = test.select_dtypes(include='category').columns.tolist()

le = LabelEncoder()

for col in cate_cols_train:
    train[col] = le.fit_transform(train[col])
    
for col in cate_cols_test:
    test[col] = le.fit_transform(test[col])

In [26]:
train = reduce_mem_usage(train)
print()
test = reduce_mem_usage(test)

Memory usage of dataframe is 2405.84 MB
Memory usage after optimization is: 957.95 MB
Decreased by 60.2%

Memory usage of dataframe is 2127.31 MB
Memory usage after optimization is: 843.14 MB
Decreased by 60.4%


### Saving cleaned and encoded Train and Test data file

In [28]:
# Saving the encoded data-files
train.to_csv('../train.csv', index = False)
test.to_csv('../test.csv', index = False)